In [ ]:
# use time-travel in langgraph
# 1. run graph with invoke and stream
# 2. identify checkpoint in existing thread using get_state_history(), locate thread_id and checkpoint_id
# 3. if need then update graph state
# 4. resume execution from checkpoint

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", api_key=GEMINI_API_KEY)

In [11]:
from langgraph.graph import StateGraph, MessagesState
from typing import TypedDict
from langgraph.constants import START, END
from langgraph.checkpoint.memory import InMemorySaver

checkpoint = InMemorySaver()

class CustomState(TypedDict):
    topic: str
    joke: str

def generate_topic(state: CustomState)->CustomState:
    response = llm.invoke("give me topic for joke, only one word and one topic")
    return {"topic": response.content}

def generate_joke(state: CustomState)->CustomState:
    response = llm.invoke(f"give me joke about {state['topic']}, under 50 words")
    return {"joke":response.content}

workflow = (
    StateGraph(CustomState)
    .add_node("generate_topic", generate_topic)
    .add_node("generate_joke", generate_joke)
    .set_entry_point("generate_topic")
    .add_edge("generate_topic", "generate_joke")
    .add_edge("generate_joke", END)
    .compile(checkpointer=checkpoint)
)

import uuid

config = {
    "configurable":{
        "thread_id": str(uuid.uuid4())
    }
}

config

{'configurable': {'thread_id': '5e64bc9f-3fd7-4577-a866-e67a6a6f821a'}}

In [12]:
response = workflow.invoke({}, config=config)
response

{'topic': 'Sleep',
 'joke': 'What do you call a sleeping dinosaur?\n\nA dino-snore!'}

In [17]:
states = list(workflow.get_state_history(config))
for state in states:
    print(state.next)
    print(state.config["configurable"]["checkpoint_id"])
    print()

()
1f07374b-c66e-674e-8002-84bfe8000792

('generate_joke',)
1f07374b-afe5-694c-8001-f66a33941a8d

('generate_topic',)
1f07374b-a7a2-63b3-8000-492b4463613e

('__start__',)
1f07374b-a7a0-69fb-bfff-43383b73653c



In [18]:
selected_state = states[1]
print(selected_state.next)
print(selected_state.values)

('generate_joke',)
{'topic': 'Sleep'}


In [19]:
new_config = workflow.update_state(selected_state.config, values={"topic":"car"})
new_config

{'configurable': {'thread_id': '5e64bc9f-3fd7-4577-a866-e67a6a6f821a',
  'checkpoint_ns': '',
  'checkpoint_id': '1f073750-7218-6a7f-8002-e815457c8408'}}

In [20]:
response = workflow.invoke(None, new_config)
response

{'topic': 'car',
 'joke': 'Why did the car get a flat tire?\nBecause it saw a fork in the road!'}